In [1]:
# convert the original popper ILP to relational datasets
import sys
import os
import re
import numpy as np
import pandas as pd

output_folder = "relational"

In [5]:
# first do the labels

labels = {"person1": [],"person2":[],"class": []}
with open("originalILP/exs.pl",'r') as f:
    lines = f.readlines()
    
    pattern = r'(pos|neg)\(f\(([\w,]+)\)\)'
    
    for line in lines:
        match = re.match(pattern, line)
        if match:
            names = match.group(2).split(",")
            labels["person1"].append(names[0])
            labels["person2"].append(names[1])
            labels["class"].append(match.group(1))

df = pd.DataFrame(labels)
df.to_csv(os.path.join(output_folder, "imdb.csv"), index=False)

In [8]:
# select 200 examples, 100 positive and 100 negative
df = df.groupby("class").apply(lambda x: x.sample(n=100)).reset_index(drop=True)
df.to_csv(os.path.join(output_folder, "imdb.csv"), index=False)

In [9]:
# bk file

kb = {}
with open("originalILP/bk.pl",'r') as f:
    lines = f.readlines()

    pattern = r"(\w+)\(([^)]+)\)"

    for line in lines:
        match = re.match(pattern, line)
        if match:
            predicate = match.group(1)
            args = match.group(2).split(',')
            if predicate not in kb:
                kb[predicate] = []
            kb[predicate].append(args)

In [10]:

# turn the kb into dataframes

for predicate in kb:
    if predicate == "movie":
        columns = ["movie","person"]
    elif predicate == "gender":
        columns = ["person","gender"]
    elif predicate == "genre":
        columns = ["person","genre"]
    elif predicate in ["director","actor"]:
        columns = ["name"]
    else:
        continue
    df = pd.DataFrame(kb[predicate], columns = columns)
    df.to_csv(f"{output_folder}/{predicate}.csv", index = False)